In [1]:
import os
from ultralytics import YOLO
import yaml
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import shutil

def verify_paths(paths_dict):
    """
    주어진 경로들이 실제로 존재하는지 확인
    """
    for name, path in paths_dict.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"{name} not found at: {path}")
        print(f"Found {name} at: {path}")

def setup_directory_structure(base_path, splits=['train', 'valid', 'test']):
    """
    Sets up the required directory structure for YOLOv8
    """
    for split in splits:
        # Create images directory
        images_dir = os.path.join(base_path, split, 'images')
        os.makedirs(images_dir, exist_ok=True)
        
        # Create labels directory
        labels_dir = os.path.join(base_path, split, 'labels')
        os.makedirs(labels_dir, exist_ok=True)
        
        # Move images to images directory
        src_dir = os.path.join(base_path, split)
        for file in os.listdir(src_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                src_path = os.path.join(src_dir, file)
                dst_path = os.path.join(images_dir, file)
                if not os.path.exists(dst_path):
                    shutil.move(src_path, dst_path)
                    print(f"Moved {file} to {images_dir}")

def extract_class_names_from_all_jsons(train_json, valid_json, test_json):
    """
    Extracts unique class names from all JSON annotation files
    """
    all_classes = set()
    
    # Process all JSON files
    for json_path in [train_json, valid_json, test_json]:
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Extract classes from current JSON using class_name
            for image_data in data.values():
                for annotation in image_data:
                    all_classes.add(annotation['class_name'])
                    
        except Exception as e:
            print(f"Warning: Error processing {json_path}: {str(e)}")
    
    # Sort classes for consistent ordering
    return sorted(list(all_classes))

def create_data_yaml(base_path, num_classes, names, yaml_path):
    """
    Creates a YAML file for YOLOv8 training configuration
    """
    data = {
        'path': base_path,  # Dataset root dir
        'train': os.path.join('train', 'images'),  # Train images relative path
        'val': os.path.join('valid', 'images'),    # Val images relative path
        'test': os.path.join('test', 'images'),    # Test images relative path
        'nc': num_classes,    # Number of classes
        'names': names        # Class names
    }
    
    # Save YAML file
    with open(yaml_path, 'w', encoding='utf-8') as f:
        yaml.dump(data, f, default_flow_style=False, allow_unicode=True)
    
    print(f"Created YAML file at: {yaml_path}")
    print(f"Number of classes: {num_classes}")
    print("Classes in order:")
    for idx, name in enumerate(names):
        print(f"{idx}: {name}")

def convert_yolo_format(json_path, output_dir, class_to_idx):
    """
    Converts JSON annotations to YOLO format using class_name
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Count successful conversions
        converted_count = 0
        processed_files = set()
        
        # Convert each image's annotations
        for image_name, annotations in data.items():
            txt_path = os.path.join(output_dir, Path(image_name).stem + '.txt')
            
            with open(txt_path, 'w', encoding='utf-8') as f:
                for ann in annotations:
                    try:
                        # Get class index from class_name (not using class_id)
                        class_name = ann['class_name']
                        class_idx = class_to_idx[class_name]
                        
                        # Get bbox coordinates (already in YOLO format)
                        x_center, y_center, width, height = ann['bbox']
                        
                        # Write to file
                        f.write(f"{class_idx} {x_center} {y_center} {width} {height}\n")
                        converted_count += 1
                        processed_files.add(image_name)
                        
                    except KeyError as e:
                        print(f"Warning: Class {ann.get('class_name', 'unknown')} not found in class mapping")
                    except Exception as e:
                        print(f"Warning: Error processing annotation in {image_name}: {str(e)}")
        
        print(f"Successfully converted {converted_count} annotations for {len(processed_files)} images to: {output_dir}")
        
    except Exception as e:
        print(f"Error processing {json_path}: {str(e)}")

def plot_metrics(results_dict, save_dir):
    """
    Plots training metrics and saves the figures
    """
    metrics = ['precision', 'recall', 'mAP50', 'mAP50-95']
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 2, i)
        plt.plot(results_dict[metric], label=f'all classes {metric}')
        plt.title(f'Training {metric}')
        plt.xlabel('epoch')
        plt.ylabel(metric)
        plt.grid(True)
        plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'training_metrics.png'))
    plt.close()

def save_metrics_csv(results_dict, save_dir):
    """
    Saves metrics to a CSV file
    """
    metrics_df = pd.DataFrame({
        'epoch': range(1, len(results_dict['precision']) + 1),
        'precision': results_dict['precision'],
        'recall': results_dict['recall'],
        'mAP50': results_dict['mAP50'],
        'mAP50-95': results_dict['mAP50-95']
    })
    
    csv_path = os.path.join(save_dir, 'training_metrics.csv')
    metrics_df.to_csv(csv_path, index=False)
    print(f"Metrics saved to: {csv_path}")
    
    # Print final metrics
    final_metrics = metrics_df.iloc[-1]
    print("\nFinal Metrics:")
    print(f"Precision: {final_metrics['precision']:.4f}")
    print(f"Recall: {final_metrics['recall']:.4f}")
    print(f"mAP50: {final_metrics['mAP50']:.4f}")
    print(f"mAP50-95: {final_metrics['mAP50-95']:.4f}")

def main():
    # Define paths
    base_path = r"C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco"
    train_path = os.path.join(base_path, "train")
    valid_path = os.path.join(base_path, "valid")
    test_path = os.path.join(base_path, "test")
    
    train_json = os.path.join(train_path, "train_annotations_yolo.json")
    valid_json = os.path.join(valid_path, "valid_annotations_yolo.json.json")
    test_json = os.path.join(test_path, "test_annotations_yolo.json")
    
    # Verify all paths exist
    paths_to_verify = {
        'Train Directory': train_path,
        'Valid Directory': valid_path,
        'Test Directory': test_path,
        'Train JSON': train_json,
        'Valid JSON': valid_json,
        'Test JSON': test_json
    }
    
    print("Verifying paths...")
    verify_paths(paths_to_verify)
    
    # Create results directory
    results_dir = os.path.join(base_path, f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    os.makedirs(results_dir, exist_ok=True)
    
    # Setup directory structure and move images
    print("Setting up directory structure...")
    setup_directory_structure(base_path)
    
    # Extract classes from all datasets
    print("\nExtracting classes from all datasets...")
    class_names = extract_class_names_from_all_jsons(train_json, valid_json, test_json)
    print(f"\nFound {len(class_names)} unique classes across all datasets:")
    
    # Create class mapping using class names
    class_to_idx = {name: i for i, name in enumerate(class_names)}
    for name, idx in class_to_idx.items():
        print(f"{idx}: {name}")
    
    # Convert annotations to YOLO format
    print("\nConverting annotations to YOLO format...")
    train_labels_dir = os.path.join(train_path, "labels")
    valid_labels_dir = os.path.join(valid_path, "labels")
    test_labels_dir = os.path.join(test_path, "labels")
    
    convert_yolo_format(train_json, train_labels_dir, class_to_idx)
    convert_yolo_format(valid_json, valid_labels_dir, class_to_idx)
    convert_yolo_format(test_json, test_labels_dir, class_to_idx)
    
    # Create data.yaml with the complete class list
    print("\nCreating data.yaml file...")
    yaml_path = os.path.join(base_path, "data.yaml")
    create_data_yaml(base_path, len(class_names), class_names, yaml_path)
    
    # Verify model file exists
    model_path = os.path.join(base_path, "yolov8n.pt")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at: {model_path}")
    print(f"\nFound model file: {model_path}")
    
    # Initialize and train YOLOv8 model
    print("\nInitializing YOLOv8 model...")
    model = YOLO(model_path)
    
    # Train the model with CPU configuration
    print("\nStarting training with CPU configuration...")
    try:
        results = model.train(
            data=yaml_path,
            epochs=100,
            imgsz=640,
            batch=8,  # Reduced batch size for CPU
            name='waste_detection_model',
            device='cpu',  # Explicitly use CPU
            project=results_dir,
            exist_ok=True,
            workers=4,  # Number of CPU workers
            patience=5  # Early stopping patience set to 5
        )
        
        # Extract metrics from training results
        metrics_dict = {
            'precision': results.results_dict['metrics/precision(B)'],
            'recall': results.results_dict['metrics/recall(B)'],
            'mAP50': results.results_dict['metrics/mAP50(B)'],
            'mAP50-95': results.results_dict['metrics/mAP50-95(B)']
        }
        
        # Plot and save metrics
        plot_metrics(metrics_dict, results_dir)
        save_metrics_csv(metrics_dict, results_dir)
        
        # Evaluate the model on test set
        print("\nEvaluating model on test set...")
        val_results = model.val(
            data=yaml_path,
            split='test',
            device='cpu'  # Explicitly use CPU for validation
        )
        
        # Save test set metrics
        test_metrics = {
            'precision': val_results.results_dict['metrics/precision(B)'],
            'recall': val_results.results_dict['metrics/recall(B)'],
            'mAP50': val_results.results_dict['metrics/mAP50(B)'],
            'mAP50-95': val_results.results_dict['metrics/mAP50-95(B)']
        }
        
        print("\nTest Set Metrics:")
        for metric, value in test_metrics.items():
            print(f"{metric}: {value:.4f}")
        
        # Save test metrics to file
        test_metrics_path = os.path.join(results_dir, 'test_metrics.txt')
        with open(test_metrics_path, 'w') as f:
            f.write("Test Set Metrics:\n")
            for metric, value in test_metrics.items():
                f.write(f"{metric}: {value:.4f}\n")
        
        # Run inference on test images with class names displayed
        print("\nRunning inference on test images...")
        model.predict(
            source=os.path.join(base_path, "test", "images"),
            save=True,
            save_txt=True,
            conf=0.25,
            project=results_dir,
            name='inference',
            boxes=True,
            labels=True,
            line_width=2
        )
        
        print("\nTraining and evaluation completed successfully!")
        print(f"Results saved in: {results_dir}")
        print(f"Inference results with class names saved in: {os.path.join(results_dir, 'inference')}")
        
    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Verifying paths...
Found Train Directory at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train
Found Valid Directory at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid
Found Test Directory at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test
Found Train JSON at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\train_annotations_yolo.json
Found Valid JSON at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\valid_annotations_yolo.json.json
Found Test JSON at: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\test\test_annotations_yolo.json
Setting up directory structure...
Moved -1-_jpg.rf.c55c25f234fe31135bf59aefc000253d.jpg to C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\images
Moved -10-_jpg.rf.986b91f3294b8704eb0ad3fe4e0487f8.jpg to C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\images
Moved -11-_jpg.rf.a9687e04d5ac30f

train: Scanning C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\labels... 636 images, 2 backgrounds, 0 corrupt: 100%|██████████| 638/638 [00:00<00:00, 1118.79it/s]

train: New cache created: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\labels.cache



val: Scanning C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\labels... 79 images, 0 backgrounds, 0 corrupt: 100%|██████████| 79/79 [00:00<00:00, 1040.08it/s]

val: New cache created: C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\valid\labels.cache


...

                   all         79        333      0.538      0.132      0.137     0.0708
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 29, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

34 epochs completed in 0.716 hours.


Optimizer stripped from C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241122_201936\waste_detection_model\weights\last.pt, 5.6MB
Optimizer stripped from C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241122_201936\waste_detection_model\weights\best.pt, 5.6MB

Validating C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241122_201936\waste_detection_model\weights\best.pt...
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i7-12700K)
Model summary (fused): 186 layers, 2,693,533 parameters, 0 gradients, 6.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.63it/s]


                   all         79        333      0.487      0.153      0.137     0.0766
              Aquarium          1          1          0          0          0          0
               Bathtub          1          1          1          0          0          0
                   Bed          4          4      0.133       0.25     0.0685     0.0406
         Bedding items          6          7          1          0        0.1     0.0678
              Bookcase          2          4          0          0          0          0
             Bookstand          2          2          1          0          0          0
               Cabinet          2          4          1          0     0.0137     0.0123
              CleanNet         47         87      0.799      0.912       0.92      0.664
   Clothes drying rack          1          1          0          0          0          0
                  Desk          1          1          0          0          0          0
          Dining tabl

TypeError: object of type 'numpy.float64' has no len()